<a href="https://colab.research.google.com/github/umbertogagl97/Tesi/blob/main/Impronte/Addestramento_scanner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Init**

## Import librerie

In [5]:
import tensorflow as tf
import numpy as np

##Def variabili

In [30]:
#Scanner
scanner_name = 'DigitalPersona'

#salvataggio modello
model_save_name = 'VGG19_first_trial'
path_model_save = F"/content/gdrive/My Drive/ModelliCNN/Scanner/{scanner_name}/{model_save_name}" 

#load dataset training
pathDataset=F'/content/gdrive/MyDrive/Dataset_impronte/training/{scanner_name}'

##Parametri

In [2]:
batch_size=128
num_class=2

##Collegamento google drive

In [3]:
#collegamento google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#**Dataset**

##Caricamento dataset

In [6]:
train_ds= tf.keras.utils.image_dataset_from_directory(
    pathDataset,
    label_mode='categorical')

Found 2000 files belonging to 2 classes.


##Calcolo dimensioni dataset

In [ ]:
#cambia: usa conteggio file nella cartella per velocizzare
dim_set=0
for image_batch, labels_batch in train_ds:
  dim_set=dim_set+image_batch.get_shape()[0]
print(dim_set)  

##Calcolo dimensioni immagini

In [10]:
image_batch, labels_batch = next(iter(train_ds))
im_shape=image_batch[0].shape

##Creazione matrici

In [11]:
x_train = np.zeros((dim_set, im_shape[0], im_shape[1], im_shape[2]), dtype=np.uint8)
y_train = np.zeros((dim_set,num_class), dtype=np.uint8)

In [12]:
x_train.shape

(2000, 256, 256, 3)

In [14]:
i=0
j=0
for image_batch, labels_batch in train_ds:
  j=i+image_batch.get_shape()[0]
  #print("i: "+str(i))
  #print("j: "+str(j))
  x_train[i : j,:,:,:]=image_batch
  y_train[i : j ,:]=labels_batch
  i=j

del i,j  

##Preprocessing immagini

In [16]:
min_, max_ = np.amin(x_train), np.amax(x_train)
x_train = (x_train - min_) / (max_ - min_)

#**Creazione modello**

###Load model pre-trained

In [21]:
#creazione rete usando vgg16 preaddestrata e aggiungendo gli ultimi livelli per adattarla al problema di 10 classi
from tensorflow.keras import applications as applications

#importa rete vgg16 addestrata sul dataset imagenet, esclude gli ultimi livelli e come input pongo dimensioni 96,96,3 
model_conv = applications.VGG19(weights='imagenet', include_top=False, input_shape=(im_shape[0],im_shape[1],im_shape[2]))

# Freeze all the layers (non modifico i pesi dei livelli inferiori)
for layer in model_conv.layers[:]:
    layer.trainable = False

80150528/80134624 [==============================] - 1s 0us/step


#**Training**
> per riaddestrare eseguire solo questa sezione



##Aggiunta classificatore

In [27]:
from tensorflow.keras import layers,models,losses,optimizers,applications
import numpy as np

# creo un modello aggiungendo livelli alla rete importata
model = models.Sequential()
# Add the vgg convolutional base model
model.add(model_conv)
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(num_class, activation='softmax')) #10: numero di valori in uscita (le classi)

#stampa info del modello
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 8, 8, 512)         20024384  
_________________________________________________________________
flatten_3 (Flatten)          (None, 32768)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 1024)              33555456  
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 2050      
Total params: 53,581,890
Trainable params: 33,557,506
Non-trainable params: 20,024,384
_________________________________________________________________


##Configura parametri modello

In [28]:
# Configure the model for training (setta i parametri del modello)
model.compile(
        loss=losses.categorical_crossentropy, optimizer=optimizers.Adam(learning_rate=1e-5), metrics=["accuracy"]
    )

##Addestramento

In [29]:
history= model.fit(x_train,y_train,epochs=10,batch_size=64)

Epoch 1/10
32/32 [==============================] - 23s 675ms/step - loss: 0.6030 - accuracy: 0.6980
Epoch 2/10
32/32 [==============================] - 22s 674ms/step - loss: 0.2657 - accuracy: 0.9015
Epoch 3/10
32/32 [==============================] - 22s 674ms/step - loss: 0.1831 - accuracy: 0.9375
Epoch 4/10
32/32 [==============================] - 22s 673ms/step - loss: 0.1487 - accuracy: 0.9515
Epoch 5/10
32/32 [==============================] - 22s 674ms/step - loss: 0.1245 - accuracy: 0.9600
Epoch 6/10
32/32 [==============================] - 22s 672ms/step - loss: 0.1037 - accuracy: 0.9725
Epoch 7/10
32/32 [==============================] - 21s 671ms/step - loss: 0.0907 - accuracy: 0.9745
Epoch 8/10
32/32 [==============================] - 22s 678ms/step - loss: 0.0832 - accuracy: 0.9805
Epoch 9/10
32/32 [==============================] - 22s 678ms/step - loss: 0.0693 - accuracy: 0.9830
Epoch 10/10
32/32 [==============================] - 22s 678ms/step - loss: 0.0617 - accura

#**Salvataggio modello**

In [31]:
#salva modello su drive
model.save(path_model_save)

INFO:tensorflow:Assets written to: /content/gdrive/My Drive/ModelliCNN/Scanner/DigitalPersona/VGG19_first_trial/assets
